In [ ]:
import sys
sys.path.append("/Users/labneuro2/.local/lib/python3.12/site-packages/network-portrait-divergence")

import os
import numpy as np
import pandas as pd
import nibabel as nib
import networkx as nx
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from portrait_divergence import portrait_divergence_weighted

from joblib import Parallel, delayed

In [ ]:
# Paths to data
data_path = "/Users/labneuro2/Documents/lab/SBvsMB4/halfpipe"
atlases_path = "/Users/labneuro2/Documents/lab/SBvsMB4/atlases"

# Atlas variants to process
atlas_variants = ['100Parcels_S1', '200Parcels_S2', '400Parcels_S4']

# Threshold for bad BOLD signal
bold_threshold = 0.8

df = pd.read_csv('dof_summary.csv', sep=';')

# Extract subject/session/task info
df['subject'] = df['file'].str.extract(r'(sub-\d+)_')[0]
df['session'] = df['file'].str.extract(r'(ses-\d+)_')[0]
df['task'] = df['file'].str.extract(r'task-([A-Za-z0-9]+)_')[0]

df_unique = df[['subject', 'session', 'task']].drop_duplicates()

In [ ]:
def load_atlas_labels(atlas_labels_file):
    with open(atlas_labels_file, 'r') as f:
        lines = f.readlines()
    labels = []
    for i in range(0, len(lines), 2):
        name = lines[i].strip()
        values = list(map(int, lines[i+1].strip().split()))
        labels.append([name] + values)
    return pd.DataFrame(labels, columns=['name', 'index', 'R', 'G', 'B', 'A'])

In [ ]:
for variant in atlas_variants:
    parcels, tian = variant.split("_")
    print(f"\n🔍 Processing atlas: {variant}")

    atlas_filename = f"{atlases_path}/Schaefer2018_{parcels}_7Networks_order_Tian_Subcortex_{tian}_3T_MNI152NLin2009cAsym_2mm.nii.gz"
    atlas_labels_file = f'{atlases_path}/Schaefer2018_{parcels}_7Networks_order_Tian_Subcortex_{tian}_label.txt'
    
    atlas_labels = load_atlas_labels(atlas_labels_file)
    all_rois = set(atlas_labels['index'])
    
    masker = NiftiLabelsMasker(atlas_filename, standardize=False)

    global_bad_rois = set()
    
    # Iterate over all unique subject/session/task combinations
    for _, row in df_unique.iterrows():
        subject = row['subject']
        session = row['session']
        task = row['task']
        
        bold_path = f'{data_path}/{subject}/{session}/func/{subject}_{session}_task-{task}_setting-preproc_desc-brain_mask.nii.gz'
        if not os.path.exists(bold_path):
            print(f"⚠️ Missing file: {bold_path}")
            continue

        try:
            # Extract time series for each ROI
            time_series = masker.fit_transform(nib.load(bold_path))
            # Mark ROIs as bad if their minimum signal is below threshold
            bad_rois = {i for i in range(time_series.shape[1]) if np.min(time_series[:, i]) < bold_threshold}
            global_bad_rois.update(bad_rois)
        except Exception as e:
            print(f"⚠️ Failed to process {bold_path}: {e}")

    good_rois = sorted(list(all_rois - global_bad_rois))
    bad_rois = sorted(list(global_bad_rois))

    # Save good and bad ROI indices to text files
    with open(f'good_rois_{variant}11.txt', 'w') as f:
        f.write('\n'.join(map(str, good_rois)))

    print(f"✅ Saved: {len(good_rois)} good and {len(bad_rois)} bad ROIs for {variant}")

In [ ]:
def correlation_matrix_to_graph(matrix, selected_rois, good_rois, threshold=0.2):
    import networkx as nx
    G = nx.Graph()
    filtered_rois = sorted(set(good_rois).intersection(selected_rois))
    matrix = matrix[np.ix_(filtered_rois, filtered_rois)]
    edges = []

    for idx_i, i in enumerate(filtered_rois):
        for idx_j, j in enumerate(filtered_rois):
            if idx_j > idx_i:
                weight = matrix[idx_i, idx_j]
                if weight > 0:
                    edges.append((i, j, weight))

    edges = sorted(edges, key=lambda x: abs(x[2]), reverse=True)
    top_percent = int(len(edges) * threshold)
    edges = edges[:top_percent]

    for i, j, weight in edges:
        G.add_edge(i, j, weight=weight)

    return G

In [ ]:
def run_portrait_divergence_analysis(
    atlas_variant="400_S4", 
    fd_threshold = 0.4,
    min_dof=15,
    roi_type="cortical", 
    trimmed_range=range(5, 14),
):
    
    parcels, tian = atlas_variant.split("_")
    atlas_prefix = f"schaefer_{parcels}_Tian_{tian}"

    fd_label = f"fd_{int(fd_threshold * 1000):03d}"
    base_path = f"{correlation_matrices_path}/{atlas_prefix}_{fd_label}"

    good_rois = np.loadtxt(f"good_rois_{atlas_variant}.txt", dtype=int)

    # Wczytaj etykiety
    atlas_labels_file = f"{atlases_path}/Schaefer2018_{parcels}_7Networks_order_Tian_Subcortex_{tian}_label.txt"
    atlas_labels = load_atlas_labels(atlas_labels_file)

    if roi_type == "cortical":
        selected_rois = set(atlas_labels[atlas_labels['name'].str.contains("Networks")]['index']-1)
    else:
        selected_rois = set(atlas_labels['index']-1)
        
    df_filtered = df[df['fd_threshold'] == fd_threshold].copy()
    df_filtered = df_filtered.dropna(subset=['afni_dof'])


    all_divergence_results = []

    for trimmed in trimmed_range:
        for method in ["SB", "MB4"]:
            subject_sessions = {}
            for subject in df_filtered['subject'].unique():
                sub_df = df_filtered[(df_filtered['subject'] == subject) & (df_filtered['task'] == method)]
                ses1 = sub_df[sub_df['session'] == 'ses-1']
                ses2 = sub_df[sub_df['session'] == 'ses-2']
                
                ses1_match = ses1[ses1['minutes'] == trimmed]
                ses2_match = ses2[ses2['minutes'] == trimmed]

                if not ses1_match.empty and not ses2_match.empty:
                    dof1 = ses1_match.iloc[0]['afni_dof']
                    dof2 = ses2_match.iloc[0]['afni_dof']
                    if dof1 >= min_dof and dof2 >= min_dof:
                        file1 = f"{base_path}/{subject}_ses-1_task-{method}_trimmed_{trimmed}min_correlation.csv"
                        file2 = f"{base_path}/{subject}_ses-2_task-{method}_trimmed_{trimmed}min_correlation.csv" 
                        if os.path.exists(file1) and os.path.exists(file2):
                            subject_sessions[subject] = (file1, file2)
            def compute_subject_divergence(subject, f1, f2, trimmed, method, selected_rois, good_rois):
                try:
                    matrix1 = np.loadtxt(f1, delimiter=',')
                    matrix2 = np.loadtxt(f2, delimiter=',')
                    G1 = correlation_matrix_to_graph(matrix1, selected_rois, good_rois)
                    G2 = correlation_matrix_to_graph(matrix2, selected_rois, good_rois)
                    div = portrait_divergence_weighted(G1, G2)
                    return (trimmed, subject, div, method)
                except Exception as e:
                    print(f"⚠️ Błąd dla {subject}: {e}")
                    return None
            
            results = Parallel(n_jobs=5)(
                delayed(compute_subject_divergence)(subject, f1, f2, trimmed, method, selected_rois, good_rois)
                for subject, (f1, f2) in subject_sessions.items()
            )
            
            divergence_results = [r for r in results if r is not None]
            all_divergence_results.extend(divergence_results)

    df_results = pd.DataFrame(all_divergence_results, columns=["Trimmed", "Subject", "Divergence", "Method"])

    return df_results

In [ ]:
correlation_matrices_path = '/Users/labneuro2/Documents/lab/SBvsMB4/correlation_matrices'

# Option lists
roi_types = ["cortical", "all"]
fd_thresholds = [0.4, 1.0]

# Output Excel file path
output_excel = "portriat_divergence_results.xlsx"

# Use a single ExcelWriter for all results
with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    for atlas_variant in atlas_variants:
        schaefer = atlas_variant.split("_")[0][:-7]
        tian = atlas_variant.split("_")[1]
        for fd_threshold in fd_thresholds:
            for roi_type in roi_types:
                if roi_type == "cortical":  
                    sheet_name = f"Schaefer{schaefer}_fd_{fd_threshold}"
                else:
                    sheet_name = f"Schaefer{schaefer}_Tian{tian}_fd_{fd_threshold}"
                print(f"\n Analysing: {atlas_variant} | {fd_threshold} | {roi_type}")
                try:
                    # Run the analysis
                    df_results = run_portrait_divergence_analysis(
                        atlas_variant=atlas_variant,
                        fd_threshold=fd_threshold,
                        min_dof=15,
                        roi_type=roi_type,
                        trimmed_range=range(5, 14)
                    )

                    # Save results to Excel sheet
                    df_results.to_excel(writer, sheet_name=sheet_name, index=False)

                except Exception as e:
                    print(f"Error {atlas_variant} + {fd_threshold} + {roi_type}: {e}")

print("\n✅ All analyses completed and Excel file saved!")
